# **Projeto de Bioinformática e Análise de Dados - Descoberta Computacional de Fármacos**

## **Download de Dados de Bioatividade**

Neste Jupyter notebook, construímos um modelo de aprendizado de máquina usando os dados de bioatividade do ChEMBL.

Realizaremos a Coleta e Pré-processamento de Dados do Banco de Dados ChEMBL.

---


## **Banco de Dados ChEMBL**

O [*Banco de Dados ChEMBL*](https://www.ebi.ac.uk/chembl/) é um banco de dados que contém dados curados de bioatividade de mais de 2 milhões de compostos. Ele é compilado a partir de mais de 76.000 documentos, 1,2 milhões de ensaios e os dados abrangem 13.000 alvos, 1.800 células e 33.000 indicações.


## **Instalando as bibliotecas**

Instalação do pacote de serviço web ChEMBL para que possamos recuperar dados de bioatividade do banco de dados ChEMBL.

In [ ]:
! pip install chembl_webresource_client

## **Importando as bibliotecas**

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Busca por proteína alvo**

### **Busca de alvo para dengue**

In [17]:
target = new_client.target
target_query = target.search('dengue')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Dengue virus,Dengue virus,15.0,False,CHEMBL613757,[],ORGANISM,12637
1,[],dengue virus type 4,dengue virus type 4,11.0,False,CHEMBL613728,[],ORGANISM,11070
2,[],dengue virus type 1,dengue virus type 1,11.0,False,CHEMBL613360,[],ORGANISM,11053
3,[],dengue virus type 2,dengue virus type 2,11.0,False,CHEMBL613966,[],ORGANISM,11060
4,[],dengue virus type 3,dengue virus type 3,11.0,False,CHEMBL612717,[],ORGANISM,11069
5,"[{'xref_id': 'P29990', 'xref_name': None, 'xre...",Dengue virus type 2 (strain Thailand/16681/198...,Dengue virus type 2 NS3 protein,9.0,False,CHEMBL5980,"[{'accession': 'P29990', 'component_descriptio...",SINGLE PROTEIN,31634


**Seleciona e recupere dados de bioatividade para *proteinase semelhante a dengue* (quinta entrada)**

O tipo 3 (DENV-3), é o mais comum no Brasil nos últimos 15 anos, apresenta maior virulência o que significa que causa sintomas mais graves que os demais.

Atribuiremos a quinta entrada (que corresponde à proteína alvo, *proteinase semelhante ao dengue virus tipo 3*) à variável ***selected_target***

In [ ]:
selected_target = targets.target_chembl_id[4]
selected_target

Aqui, recuperaremos apenas dados de bioatividade para *proteinase semelhante ao dengue virus tipo 3* (CHEMBL612717) que são relatados como valores de IC$_{50}$ em unidades nM (nanomolar).

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df.head(3)

Por fim, salvaremos os dados de bioatividade resultantes em um arquivo CSV **bioactivity_data.csv**.

In [19]:
df.to_csv('dengue_bioactivity_data.csv', index=False)

## **Manipulando dados ausentes**
Se algum composto tiver valor ausente para a coluna **standard_value**, então descarte-o

In [ ]:
df2 = df[df.standard_value.notna()]
df2

Aparentemente, para este conjunto de dados não há dados faltantes. Mas podemos usar a célula de código acima para dados de bioatividade de outra proteína alvo.

## **Pré-processamento de dados de bioatividade**

### **Rotulagem de compostos como ativos, inativos ou intermediários**
Os dados de bioatividade estão na unidade IC50. Compostos com valores menores que 1000 nM serão considerados **ativos**, enquanto aqueles maiores que 10.000 nM serão considerados **inativos**. Já aqueles valores entre 1.000 e 10.000 nM serão chamados de **intermediários**.

In [ ]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

### **Combine as 3 colunas (molecule_chembl_id,canonical_smiles,standard_value) e bioactivity_class em um DataFrame**

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2[selection]
df3

In [ ]:
bioactivity_class = pd.Series(bioactivity_class, name='bioactivity_class')
df4 = pd.concat([df3, bioactivity_class], axis=1)
df4

Salva o dataframe em um arquivo CSV

In [ ]:
df4.to_csv('dengue_bioactivity_data_preprocessed.csv', index=False)

In [ ]:
! ls -l

---